In [1]:
import os

os.chdir('../src/')

In [2]:
from modules.dataProcessing import getJoinedData

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
jumboData = getJoinedData( 
                cachedFile='../data/intermediate/jumboData.snappy.parquet'
            )   ## Data will be fetched if cached file is not found

Below is to check what are the regions associated with country == 'reserved/private'

In [4]:
jumboData[ jumboData['country'] == 'reserved/private' ]['region'].value_counts()

reserved/private    13
Name: region, dtype: int64

Below is to check what are the city_code associated with country == 'reserved/private'

In [5]:
jumboData[ jumboData['country'] == 'reserved/private' ]['city_code'].value_counts()

Unknown    13
Name: city_code, dtype: int64

Concluded that country='reserved/private' not need to be cleansed.

================================================================================ <br>
Assuming the null values under `goal_type` column as 3rd goal type, which is different from goal type 1 & 2.
Hence, to fill null goal_type as 3.

In [6]:
jumboData['goal_type'] = jumboData['goal_type'].fillna(3)
jumboData['goal_type'] = jumboData['goal_type'].astype(np.int)

================================================================================ <br>
Assuming the null values under `conversions` column as "converted"

In [7]:
jumboData['conversions'] = jumboData['conversions'].fillna(1)

================================================================================ <br>
Assuming the null values under `impressions` column follows the median of impressions by country and region

In [8]:
country_region_impressionNUll    = jumboData[ jumboData['impressions'].isnull() ][['country', 'region']].drop_duplicates()
country_region_impressionNotNUll = jumboData[ jumboData['impressions'].notnull() ][['country', 'region']].drop_duplicates()

In [9]:
country_region_impressionNUll    = country_region_impressionNUll.values.tolist()
country_region_impressionNotNUll = country_region_impressionNotNUll.values.tolist()

As shown belw, all distinct country and region pair with null values under `impressions` column, can be mapped by country and region pair with captured `impressions` value (in this case, median value is proposed)

In [10]:
[tup for tup in country_region_impressionNUll if tup not in country_region_impressionNotNUll ]

[]

In [11]:
impressionMapping = jumboData[ jumboData['impressions'].notnull() ].groupby(['country', 'region'])['impressions'].agg(np.median)
impressionMapping = impressionMapping.reset_index()
impressionMapping.columns = ['country', 'region', 'median_impressions']
impressionMapping[:5]

,country,region,median_impressions
0,Australia,Australian Capital Territory,7.0
1,Australia,New South Wales,4.0
2,Australia,Northern Territory,2.0
3,Australia,Queensland,3.0
4,Australia,South Australia,3.0


In [12]:
jumboData = pd.merge(jumboData, impressionMapping, on=['country', 'region'], how='left')

As shown below, impression with null values are replaced by median impression value based on the same country & region

In [13]:
jumboData['impressions'] = jumboData['impressions'].fillna( jumboData['median_impressions'] )

================================================================================ <br>
Assuming the null values under clicks column follows the median of clicks by country and region

In [14]:
country_region_clicksNUll    = jumboData[ jumboData['clicks'].isnull() ][['country', 'region']].drop_duplicates()
country_region_clicksNotNUll = jumboData[ jumboData['clicks'].notnull() ][['country', 'region']].drop_duplicates()

In [15]:
country_region_clicksNUll    = country_region_clicksNUll.values.tolist()
country_region_clicksNotNUll = country_region_clicksNotNUll.values.tolist()

As shown belw, all distinct country and region pair with null values under clicks column, can be mapped by country and region pair with captured clicks value (in this case, median value is used again)

In [16]:
[tup for tup in country_region_clicksNUll if tup not in country_region_clicksNotNUll ]

[]

In [17]:
clicksMapping = jumboData[ jumboData['clicks'].notnull() ].groupby(['country', 'region'])['clicks'].agg(np.median)
clicksMapping = clicksMapping.reset_index()
clicksMapping.columns = ['country', 'region', 'median_clicks']
clicksMapping[:5]

,country,region,median_clicks
0,Australia,Australian Capital Territory,0.0
1,Australia,New South Wales,0.0
2,Australia,Northern Territory,0.0
3,Australia,Queensland,0.0
4,Australia,South Australia,0.0


In [18]:
jumboData = pd.merge(jumboData, clicksMapping, on=['country', 'region'], how='left')

As shown below, clicks with null values are replaced by median clicks value based on the same country & region

In [19]:
jumboData['clicks'] = jumboData['clicks'].fillna( jumboData['median_clicks'] )

In [20]:
jumboData.to_parquet( '../data/intermediate/jumboData_cleansed.snappy.parquet', engine='pyarrow', compression='snappy' )